In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager  # 新增：導入 webdriver_manager
import pandas as pd
import time
import os

def handle_load_more_button(driver):
    """
    改進的「載入更多」按鈕處理邏輯，使用多種策略找到並點擊按鈕
    
    Args:
        driver: Selenium WebDriver 實例
        
    Returns:
        bool: 是否成功點擊按鈕
    """
    # 常見的「載入更多」按鈕選擇器
    load_more_selectors = [
        'button.show_more', 
        'button[data-testid="show-more-results-button"]',
        '.b6dc9a9e69',
        'button.fc63351294',  # 可能的新選擇器
        'button.moreResults',
        'button.show-more-results',
        'button[data-component="search/pagination/show-more"]'
    ]
    
    # 1. 先嘗試使用 JavaScript 根據按鈕文字查找
    js_find_button = '''
    function findButtonByText(text) {
        const buttons = document.querySelectorAll('button');
        for (let button of buttons) {
            if (button.textContent.includes(text) && button.offsetParent !== null) {
                return button;
            }
        }
        return null;
    }
    
    const loadMoreTexts = ["載入更多搜尋結果", "顯示更多結果", "載入更多", "顯示更多", "更多結果", "Load more", "Show more"];
    
    for (let text of loadMoreTexts) {
        const button = findButtonByText(text);
        if (button) {
            return button;
        }
    }
    
    return null;
    '''
    
    button_found = False
    
    try:
        load_more_button = driver.execute_script(js_find_button)
        if load_more_button:
            print("  通過文本內容找到「載入更多」按鈕")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", load_more_button)
            print("  已點擊「載入更多」按鈕")
            button_found = True
            time.sleep(3)  # 等待新內容加載
    except Exception as e:
        print(f"  使用 JavaScript 查找按鈕時出錯: {e}")
    
    # 2. 如果 JavaScript 方法失敗，嘗試使用選擇器
    if not button_found:
        for selector in load_more_selectors:
            try:
                buttons = driver.find_elements(By.CSS_SELECTOR, selector)
                visible_buttons = [b for b in buttons if b.is_displayed()]
                
                if visible_buttons:
                    print(f"  找到「載入更多」按鈕，使用選擇器: {selector}")
                    
                    # 確保按鈕在視圖中
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", visible_buttons[0])
                    time.sleep(1)
                    
                    # 嘗試點擊
                    try:
                        visible_buttons[0].click()
                        print("  已點擊「載入更多」按鈕")
                        button_found = True
                        time.sleep(3)  # 等待新內容加載
                        break
                    except:
                        # 如果直接點擊失敗，嘗試使用 JavaScript 點擊
                        driver.execute_script("arguments[0].click();", visible_buttons[0])
                        print("  已使用 JavaScript 點擊「載入更多」按鈕")
                        button_found = True
                        time.sleep(3)  # 等待新內容加載
                        break
            except Exception as e:
                continue
    
    # 3. 嘗試使用 XPath 查找包含特定文本的按鈕
    if not button_found:
        try:
            xpath_patterns = [
                "//button[contains(text(), '載入更多搜尋結果')]",
                "//button[contains(text(), '顯示更多結果')]",
                "//button[contains(text(), '載入更多')]",
                "//button[contains(text(), '顯示更多')]",
                "//button[contains(text(), 'Load more')]",
                "//button[contains(text(), 'Show more')]"
            ]
            
            for xpath in xpath_patterns:
                buttons = driver.find_elements(By.XPATH, xpath)
                if buttons:
                    print(f"  通過 XPath 找到「載入更多」按鈕: {xpath}")
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", buttons[0])
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", buttons[0])
                    print("  已點擊「載入更多」按鈕")
                    button_found = True
                    time.sleep(3)  # 等待新內容加載
                    break
        except Exception as e:
            print(f"  使用 XPath 查找按鈕時出錯: {e}")
    
    return button_found

def scrape_booking_with_selenium(location, checkin_date, checkout_date, max_scrolls=5, chromedriver_path=None):
    # 設置 Chrome 選項
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 無頭模式
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # 初始化 WebDriver，使用 webdriver_manager 自動管理 ChromeDriver
    if chromedriver_path:
        service = Service(executable_path=chromedriver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)
    else:
        # 使用 webdriver_manager 自動下載和管理 ChromeDriver
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        # 構建 URL
        base_url = "https://www.booking.com/searchresults.zh-tw.html"
        params = {
            'ss': location,
            'checkin': checkin_date,
            'checkout': checkout_date,
            'group_adults': '2',
            'no_rooms': '1',
            'group_children': '0',
            'selected_currency': 'MYR'  # 指定貨幣
        }
        
        # 構建查詢字符串
        query_string = "&".join([f"{k}={v}" for k, v in params.items()])
        url = f"{base_url}?{query_string}"
        
        print(f"訪問 URL: {url}")
        
        # 訪問頁面
        driver.get(url)
        
        # 等待頁面加載 - 增加等待時間
        print("等待頁面加載...")
        time.sleep(10)
        
        # 保存頁面源代碼以便調試
        with open("booking_page.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print("已保存頁面源代碼到 booking_page.html")
        
        # 嘗試多個可能的選擇器
        possible_selectors = [
            'div[data-testid="property-card"]',
            'div.sr_property_block',
            'div.sr-hotel__title',
            'div.a826ba81c4',  # 有時 Booking 使用隨機類名
            'div.d4924c9e74',
            'div[data-testid="property-card-container"]'  # 新的可能選擇器
        ]
        
        found_elements = False
        selector_used = ""
        
        for selector in possible_selectors:
            print(f"嘗試選擇器: {selector}")
            try:
                # 縮短等待時間，因為我們嘗試多個選擇器
                hotel_cards = driver.find_elements(By.CSS_SELECTOR, selector)
                if hotel_cards:
                    print(f"找到 {len(hotel_cards)} 個酒店卡片，使用選擇器: {selector}")
                    found_elements = True
                    selector_used = selector
                    break
            except Exception as e:
                print(f"選擇器 {selector} 失敗: {e}")
        
        if not found_elements:
            print("無法找到任何酒店卡片元素，嘗試獲取頁面標題...")
            print(f"頁面標題: {driver.title}")
            
            # 檢查是否有反爬蟲機制啟動
            if "Booking.com: 確認您不是機器人" in driver.title or "robot" in driver.page_source.lower():
                print("檢測到反爬蟲機制，可能需要解決 CAPTCHA")
            
            return pd.DataFrame()  # 返回空 DataFrame
        
        # 執行滾動以加載更多內容
        print(f"開始滾動頁面以加載更多內容，最多滾動 {max_scrolls} 次...")
        hotels = []
        last_count = 0
        consecutive_no_new_content = 0  # 追蹤連續沒有新內容的次數
        
        for scroll in range(max_scrolls):
            print(f"執行第 {scroll + 1} 次滾動...")
            
            # 滾動到頁面底部
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # 等待內容加載
            
            # 獲取當前的酒店卡片
            hotel_cards = driver.find_elements(By.CSS_SELECTOR, selector_used)
            current_count = len(hotel_cards)
            
            print(f"當前找到 {current_count} 個酒店卡片")
            
            # 如果沒有新的酒店被加載，嘗試點擊"載入更多"按鈕
            if current_count == last_count:
                consecutive_no_new_content += 1
                print(f"連續 {consecutive_no_new_content} 次沒有新內容")
                
                # 嘗試點擊「載入更多」按鈕
                button_clicked = handle_load_more_button(driver)
                
                if not button_clicked:
                    print("沒有找到或無法點擊「載入更多」按鈕")
                    # 如果連續兩次沒有新內容且沒有加載更多按鈕，則可能已到底部
                    if consecutive_no_new_content >= 2:
                        print("似乎已經到達頁面底部，停止滾動")
                        break
            else:
                # 重置計數器，因為有新內容
                consecutive_no_new_content = 0
            
            last_count = current_count
            
            # 每次滾動後重新獲取酒店卡片，以確保我們有最新的元素
            hotel_cards = driver.find_elements(By.CSS_SELECTOR, selector_used)
            print(f"滾動後找到 {len(hotel_cards)} 個酒店卡片")
        
        # 提取酒店信息
        print(f"開始提取 {len(hotel_cards)} 個酒店的信息...")
        for i, card in enumerate(hotel_cards):
            try:
                print(f"處理酒店 #{i+1}")
                
                # 嘗試多種方式獲取名稱
                name = None
                for name_selector in ['div[data-testid="title"]', 'span.sr-hotel__name', '.fcab3ed991', '.e13098a59f']:
                    try:
                        name_elem = card.find_element(By.CSS_SELECTOR, name_selector)
                        name = name_elem.text.strip()
                        if name:
                            break
                    except:
                        continue
                
                if not name:
                    print(f"酒店 #{i+1}: 無法獲取名稱")
                    continue
                
                # 嘗試獲取價格
                price = "N/A"
                for price_selector in ['span[data-testid="price-and-discounted-price"]', '.bui-price-display__value', '.prco-valign-middle-helper', '.fcab3ed991', 'span.fcab3ed991', 'span.fde444d7ef._e885fdc12']:
                    try:
                        price_elem = card.find_element(By.CSS_SELECTOR, price_selector)
                        price = price_elem.text.strip()
                        break
                    except:
                        continue
                
                # 嘗試獲取評分
                rating = "N/A"
                for rating_selector in ['div[data-testid="review-score"] div', '.bui-review-score__badge', '.b5cd09854e', 'div._9c5f726ff', 'div._2c3a3788cd']:
                    try:
                        rating_elem = card.find_element(By.CSS_SELECTOR, rating_selector)
                        rating = rating_elem.text.strip()
                        break
                    except:
                        continue
                
                # 嘗試獲取位置
                location_info = "N/A"
                for location_selector in ['span[data-testid="address"]', '.bui-card__subtitle', '.f4bd0794db', 'span._c5d12bf22', 'span.aee5343fdb']:
                    try:
                        location_elem = card.find_element(By.CSS_SELECTOR, location_selector)
                        location_info = location_elem.text.strip()
                        break
                    except:
                        continue
                
                print(f"酒店信息: {name}, 價格: {price}, 評分: {rating}, 位置: {location_info}")
                hotels.append({
                    'name': name,
                    'price': price,
                    'rating': rating,
                    'location': location_info
                })
            except Exception as e:
                print(f"處理酒店 #{i+1} 時出錯: {e}")
        
        # 轉換為 DataFrame
        df = pd.DataFrame(hotels)
        
        # 保存結果到 CSV
        csv_filename = f"booking_{location}_{checkin_date}_{checkout_date}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"結果已保存到 {csv_filename}")
        
        return df
        
    except Exception as e:
        print(f"發生錯誤: {e}")
        return pd.DataFrame()
    finally:
        # 關閉瀏覽器
        driver.quit()

# 使用示例
if __name__ == "__main__":
    try:
        # 使用 webdriver_manager 自動管理 ChromeDriver，不需要指定路徑
        results = scrape_booking_with_selenium("Petaling Jaya", "2025-03-15", "2025-03-16", max_scrolls=20)
        if not results.empty:
            print("\n抓取結果:")
            print(results)
        else:
            print("未找到任何酒店數據")
    except Exception as e:
        print(f"運行腳本時發生錯誤: {e}")


訪問 URL: https://www.booking.com/searchresults.zh-tw.html?ss=Petaling Jaya&checkin=2025-03-15&checkout=2025-03-16&group_adults=2&no_rooms=1&group_children=0&selected_currency=MYR
等待頁面加載...
已保存頁面源代碼到 booking_page.html
嘗試選擇器: div[data-testid="property-card"]
找到 25 個酒店卡片，使用選擇器: div[data-testid="property-card"]
開始滾動頁面以加載更多內容，最多滾動 20 次...
執行第 1 次滾動...
當前找到 44 個酒店卡片
滾動後找到 44 個酒店卡片
執行第 2 次滾動...
當前找到 66 個酒店卡片
滾動後找到 66 個酒店卡片
執行第 3 次滾動...
當前找到 66 個酒店卡片
連續 1 次沒有新內容
  通過文本內容找到「載入更多」按鈕
  已點擊「載入更多」按鈕
滾動後找到 88 個酒店卡片
執行第 4 次滾動...
當前找到 88 個酒店卡片
滾動後找到 88 個酒店卡片
執行第 5 次滾動...
當前找到 88 個酒店卡片
連續 1 次沒有新內容
  通過文本內容找到「載入更多」按鈕
  已點擊「載入更多」按鈕
滾動後找到 112 個酒店卡片
執行第 6 次滾動...
當前找到 112 個酒店卡片
滾動後找到 112 個酒店卡片
執行第 7 次滾動...
當前找到 112 個酒店卡片
連續 1 次沒有新內容
  通過文本內容找到「載入更多」按鈕
  已點擊「載入更多」按鈕
滾動後找到 137 個酒店卡片
執行第 8 次滾動...
當前找到 137 個酒店卡片
滾動後找到 137 個酒店卡片
執行第 9 次滾動...
當前找到 137 個酒店卡片
連續 1 次沒有新內容
  通過文本內容找到「載入更多」按鈕
  已點擊「載入更多」按鈕
滾動後找到 160 個酒店卡片
執行第 10 次滾動...
當前找到 160 個酒店卡片
滾動後找到 160 個酒店卡片
執行第 11 次滾動...
當前找到 160 個酒店卡片
連續 1 次沒有新內容
  通過文本內容找到「載